In [229]:
import translate as tr
import pandas as pd
import numpy as np
from sklearn_pandas import DataFrameMapper
from sklearn import preprocessing
import sys
import cPickle as pickle
from datetime import date
import calendar
import scipy.io as spi
import scipy.sparse as sps
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from lightfm import LightFM
import ml_metrics as metrics

In [34]:
raw_data =  pd.read_csv('/Users/jonathangreve/Documents/listing-user-interactions.csv', names=['user_id','listing_id','make','mileage','price','year'])

In [35]:
raw_data.head(10)

,user_id,listing_id,make,mileage,price,year
0,1205,9214,chevrolet,91503.0,6115.0,2015.0
1,1258,6508,subaru,85613.0,11947.0,2017.0
2,2481,8689,plymouth,44734.0,8141.0,2015.0
3,1647,2520,mercedes-benz,51343.0,41869.0,2014.0
4,496,4904,mitsubishi,63187.0,8569.0,2016.0
5,1210,2301,dodge,72717.0,6511.0,2017.0
6,1373,6082,volvo,67771.0,18499.0,2011.0
7,1017,8433,dodge,6779.0,6365.0,2012.0
8,698,5010,honda,58110.0,5856.0,2011.0
9,614,923,toyota,50230.0,4124.0,2015.0


testing and training set

In [36]:
raw_data_train, raw_data_test = train_test_split(raw_data, train_size = 0.8)

In [141]:
raw_data.shape

(9999, 6)

In [89]:
train_users = raw_data_train.iloc[:, :1]
test_users = raw_data_test.iloc[:, :1]
train_users_unique = train_users.drop_duplicates(subset=None, keep='first', inplace=False)
test_users_unique = test_users.drop_duplicates(subset=None, keep='first', inplace=False)

In [65]:
train_users.head(5)

,user_id
6623,1795
5558,2159
3352,1941
3439,442
4044,1204


user_listing_interactions

In [145]:
train_user_listing_actions = raw_data_train.iloc[:, :2]
test_user_listing_actions = raw_data_test.iloc[:, :2]
user_listing_actions = raw_data.iloc[:, :2]

Listing-feature-matrix

In [95]:
vec = DictVectorizer(sparse=True)
train_listings = raw_data_train.iloc[:, 1:6]
train_listings_unique = train_listings.drop_duplicates(subset=None, keep='first', inplace=False)
train_listings_matrix_unique = vec.fit_transform(train_listings_unique.T.to_dict().values())
train_listings_matrix_unique = sps.csr_matrix(train_listings_matrix_unique, dtype=np.int32)
train_listings_matrix = vec.fit_transform(train_listings.T.to_dict().values())
train_listings_matrix = sps.csr_matrix(train_listings_matrix, dtype=np.int32)
test_listings = raw_data_test.iloc[:, 1:6]
test_listings_unique = test_listings.drop_duplicates(subset=None, keep='first', inplace=False)
test_listings_matrix = vec.fit_transform(test_listings.T.to_dict().values())
test_listings_matrix = sps.csr_matrix(test_listings_matrix, dtype=np.int32)
test_listings_matrix_unique = vec.fit_transform(test_listings_unique.T.to_dict().values())
test_listings_matrix_unique = sps.csr_matrix(test_listings_matrix_unique, dtype=np.int32)

In [87]:
train_listings_unique.shape

(5513, 5)

In [184]:
# Build a dict with the listing index in train_listings
d_li_tr = {}
for i in range(len(train_listings)):
    listing = train_listings["listing_id"].values[i]
    d_li_tr[listing] = i

In [185]:
len(d_li_tr)

10000

In [170]:
# Build a dict with the user index in train_users
d_ui = {}
for i in range(len(train_users_unique)):
    user = train_users_unique["user_id"].values[i]
    d_ui[user] = i


In [171]:
len(d_ui)

2772

In [178]:
# Build the user x listing matrices using scipy lil_matrix
Mui_tr = sps.lil_matrix((len(d_ui), len(d_li_tr)), dtype=np.int8)

In [179]:
Mui_tr

<2772x5513 sparse matrix of type '<type 'numpy.int8'>'
	with 0 stored elements in LInked List format>

Creation of implicit feed back user/listing matrix

In [190]:
for i in range(len(train_user_listing_actions)):
    sys.stdout.write(
        "\rProcessing row " + str(i) + "/ " + str(train_user_listing_actions.shape[0]))
    sys.stdout.flush()
    user = train_user_listing_actions["user_id"].values[i]
    listing = train_user_listing_actions["listing_id"].values[i]
    ui = d_ui[user]
    li = d_li_tr[listing]
    Mui_tr[ui, li] = 1
    print Mui_tr
print

Processing row 0/ 7999  (0, 0)	1
  (0, 435)	1
  (1, 1)	1
  (1, 1147)	1
  (1, 2224)	1
  (1, 3338)	1
  (1, 3682)	1
  (1, 3794)	1
  (2, 2)	1
  (2, 315)	1
  (3, 3)	1
  (3, 163)	1
  (3, 416)	1
  (3, 2955)	1
  (3, 4694)	1
  (4, 4)	1
  (4, 1170)	1
  (5, 5)	1
  (5, 569)	1
  (5, 3592)	1
  (6, 6)	1
  (6, 856)	1
  (6, 1630)	1
  (6, 3442)	1
  (6, 4860)	1
  (7, 7)	1
  (8, 8)	1
  (8, 79)	1
  (8, 1614)	1
  (8, 2657)	1
  (8, 3305)	1
  (9, 9)	1
  (9, 173)	1
  (9, 1918)	1
  (9, 2227)	1
  (10, 10)	1
  (10, 742)	1
  (10, 1400)	1
  (10, 1607)	1
  (10, 1892)	1
  (10, 4908)	1
  (11, 11)	1
  (11, 139)	1
  (11, 1381)	1
  (12, 12)	1
  (12, 321)	1
  (12, 2071)	1
  (12, 3766)	1
  (12, 4769)	1
  (13, 13)	1
  (13, 2858)	1
  (13, 3006)	1
  (14, 14)	1
  (14, 2815)	1
  (15, 15)	1
  (15, 198)	1
  (15, 1548)	1
  (15, 3091)	1
  (15, 3678)	1
  (16, 16)	1
  (16, 887)	1
  (16, 1099)	1
  (16, 1753)	1
  (17, 17)	1
  (17, 1437)	1
  (17, 1509)	1
  (17, 2616)	1
  (17, 5258)	1
  (18, 18)	1
  (18, 389)	1
  (18, 1890)	1
  (18, 4419

Processing row 1/ 7999  (0, 0)	1
  (0, 435)	1
  (1, 1)	1
  (1, 1147)	1
  (1, 2224)	1
  (1, 3338)	1
  (1, 3682)	1
  (1, 3794)	1
  (2, 2)	1
  (2, 315)	1
  (3, 3)	1
  (3, 163)	1
  (3, 416)	1
  (3, 2955)	1
  (3, 4694)	1
  (4, 4)	1
  (4, 1170)	1
  (5, 5)	1
  (5, 569)	1
  (5, 3592)	1
  (6, 6)	1
  (6, 856)	1
  (6, 1630)	1
  (6, 3442)	1
  (6, 4860)	1
  (7, 7)	1
  (8, 8)	1
  (8, 79)	1
  (8, 1614)	1
  (8, 2657)	1
  (8, 3305)	1
  (9, 9)	1
  (9, 173)	1
  (9, 1918)	1
  (9, 2227)	1
  (10, 10)	1
  (10, 742)	1
  (10, 1400)	1
  (10, 1607)	1
  (10, 1892)	1
  (10, 4908)	1
  (11, 11)	1
  (11, 139)	1
  (11, 1381)	1
  (12, 12)	1
  (12, 321)	1
  (12, 2071)	1
  (12, 3766)	1
  (12, 4769)	1
  (13, 13)	1
  (13, 2858)	1
  (13, 3006)	1
  (14, 14)	1
  (14, 2815)	1
  (15, 15)	1
  (15, 198)	1
  (15, 1548)	1
  (15, 3091)	1
  (15, 3678)	1
  (16, 16)	1
  (16, 887)	1
  (16, 1099)	1
  (16, 1753)	1
  (17, 17)	1
  (17, 1437)	1
  (17, 1509)	1
  (17, 2616)	1
  (17, 5258)	1
  (18, 18)	1
  (18, 389)	1
  (18, 1890)	1
  (18, 4419

Processing row 2/ 7999  (0, 0)	1
  (0, 435)	1
  (1, 1)	1
  (1, 1147)	1
  (1, 2224)	1
  (1, 3338)	1
  (1, 3682)	1
  (1, 3794)	1
  (2, 2)	1
  (2, 315)	1
  (3, 3)	1
  (3, 163)	1
  (3, 416)	1
  (3, 2955)	1
  (3, 4694)	1
  (4, 4)	1
  (4, 1170)	1
  (5, 5)	1
  (5, 569)	1
  (5, 3592)	1
  (6, 6)	1
  (6, 856)	1
  (6, 1630)	1
  (6, 3442)	1
  (6, 4860)	1
  (7, 7)	1
  (8, 8)	1
  (8, 79)	1
  (8, 1614)	1
  (8, 2657)	1
  (8, 3305)	1
  (9, 9)	1
  (9, 173)	1
  (9, 1918)	1
  (9, 2227)	1
  (10, 10)	1
  (10, 742)	1
  (10, 1400)	1
  (10, 1607)	1
  (10, 1892)	1
  (10, 4908)	1
  (11, 11)	1
  (11, 139)	1
  (11, 1381)	1
  (12, 12)	1
  (12, 321)	1
  (12, 2071)	1
  (12, 3766)	1
  (12, 4769)	1
  (13, 13)	1
  (13, 2858)	1
  (13, 3006)	1
  (14, 14)	1
  (14, 2815)	1
  (15, 15)	1
  (15, 198)	1
  (15, 1548)	1
  (15, 3091)	1
  (15, 3678)	1
  (16, 16)	1
  (16, 887)	1
  (16, 1099)	1
  (16, 1753)	1
  (17, 17)	1
  (17, 1437)	1
  (17, 1509)	1
  (17, 2616)	1
  (17, 5258)	1
  (18, 18)	1
  (18, 389)	1
  (18, 1890)	1
  (18, 4419

IndexError: column index (6706) out of bounds

In [191]:
Mui_tr

<2772x5513 sparse matrix of type '<type 'numpy.int8'>'
	with 8000 stored elements in LInked List format>

Build Model

In [192]:
no_comp = 5
lr= 0.7
ep=5

In [193]:
model = LightFM(no_components=no_comp, learning_rate=lr, loss='warp')
model.fit_partial(
    Mui_tr,
    user_features=None,
    item_features=train_listings_matrix_unique,
    epochs=ep,
    num_threads=4,
    verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [194]:
train_listings_unique.shape

(5513, 5)

In [294]:
model.predict(
        [10],
        pid_array,
        user_features=None,
        item_features=test_listings_matrix_unique,
        num_threads=4)

array([  4.08173796e+36,   1.54052063e+37,   1.25761144e+37, ...,
         1.07918430e+37,   1.26533067e+37,   6.33935522e+36])

In [299]:
list_listing = test_listings_unique["listing_id"].values
list_user = test_users_unique["user_id"].values

test = sps.csr_matrix((len(list_user),len(list_listing)),dtype=np.int32)
no_users, no_listings = test.shape
pid_array = np.arange(no_listings, dtype=np.int32)

# Create and initialise dict to store predictions
d_user_pred = {}

for user in list_user:
    d_user_pred[user] = []

# print d_user_pred
# Loop over users and compute predictions
for user_id, row in enumerate(test):
    sys.stdout.write("\rProcessing user " + str(user_id)+ "/ " + str(len(list_user)))
    sys.stdout.flush()
    uid_array = np.empty(no_listings, dtype=np.int32)
    uid_array.fill(user_id)
    predictions = model.predict(
        uid_array,
        pid_array,
        user_features=None,
        item_features=test_listings_matrix_unique,
        num_threads=4)
#     Get the top 10 recommendations for each user
    top_k = np.argsort(predictions)[:10]
    user = list_user[user_id]
    d_user_pred[user] = top_k

Processing user 1461/ 1462

In [300]:
d_user_pred[2868]

array([ 738,  609,  830,  793, 1483,  722, 1378,   53, 1145,  842])

In [258]:
# Format predictions
# for index, user in enumerate(list_user):
#     list_pred = d_user_pred[user]
#     top_k = np.argsort(list_pred)[:10]
#     d_user_pred[user] = list_listing[top_k]

In [280]:
# user_listing_actions.head(5)

In [260]:
d_user_actual = {}
for index, row in user_listing_actions.iterrows():
    d_user_actual[row['user_id']] = []
    
for index, row in user_listing_actions.iterrows():
    d_user_actual[row['user_id']].append(row['listing_id'])

In [261]:
dict_user_listing_actions={}
# Format actual
for index, user in enumerate(list_user):
    list_actual = d_user_actual[user]
    top_k = np.argsort(list_actual)[:10]
    dict_user_listing_actions[user] = list_listing[top_k]     

In [309]:
# dict_user_listing_actions

In [281]:
d_user_actions = {}
list_user = dict_user_listing_actions.keys()
list_actual = [dict_user_listing_actions[key] for key in list_user]
list_pred = [d_user_pred[key] for key in list_user]

In [310]:
# list_actual

In [311]:
# d_user_pred

In [313]:
list_score = []
for user in list_user:
    print dict_user_listing_actions[user]
    user_pred = d_user_pred[user]
    user_actual = dict_user_listing_actions[user]
    list_score.append(metrics.mapk(list_actual, list_pred))

print list_score
list_score = np.array(list_score)
print list_score
print str(np.mean(list_score)) + " +/- " + str(np.std(list_score))

[2790]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()